# Embedding
임베딩은 LLM, RAG에서 중요한 개념이다. 즉, 자연어를 기계가 이해할 수 데이터로 변환하는 작업을 임베딩이라고 한다. 
임베딩 모델은 이미 사전에 학습된 모델이며 상황에 맞는 임베딩 모델을 사용하면 된다.

In [1]:
from langchain_openai import OpenAIEmbeddings

# OpenAI의 "text-embedding-3-large" 모델을 사용하여 임베딩을 생성합니다.
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
text = "임베딩 테스트를 하기 위한 샘플 문장입니다."
query_result = embeddings.embed_query(text)
print(query_result[:5])
print(len(query_result))

[-0.00776276458054781, 0.03680367395281792, 0.019545823335647583, -0.0196656696498394, 0.017203375697135925]
1536


In [ ]:
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
import json

with open(r"C:\Users\cn120\바탕 화면\rag\fairness\data\dumy\total_articls.json", 'r', encoding='UTF-8') as f:
    json_data = json.load(f)

text_splitter = CharacterTextSplitter(
    # 텍스트를 분할할 때 사용할 구분자를 지정합니다. 기본값은 "\n\n"입니다.
    # separator= " ",
    # 분할된 텍스트 청크의 최대 크기(글자 갯수)를 지정합니다.
    chunk_size=3000,
    # 분할된 텍스트 청크 간의 중복되는 문자 수를 지정합니다.
    chunk_overlap=50,
    # 텍스트의 길이를 계산하는 함수를 지정합니다.
    length_function=len,
    # 구분자가 정규식인지 여부를 지정합니다.
    is_separator_regex=False,
)

embeddings_1024 = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=1024)
# text_splitter를 사용하여 state_of_the_union 텍스트를 문서로 분할합니다.
texts = text_splitter.create_documents([json_data[i]['기사본문'] for i in range(len(json_data))])
doc_result = embeddings_1024.embed_documents(
    [text.page_content for text in texts]
)  # 텍스트를 임베딩하여 문서 벡터를 생성합니다.
# 문서 결과의 첫 번째 요소에서 처음 5개 항목을 선택합니다.
# doc_result[1][:5]
print(len(doc_result[0]))

1024


# 유사도 계산

In [4]:
sentence1 = "안녕하세요? 반갑습니다."
sentence2 = "안녕하세요? 반갑습니다!"
sentence3 = "안녕하세요? 만나서 반가워요."
sentence4 = "Hi, nice to meet you."
sentence5 = "I like to eat apples."

from langchain_openai import OpenAIEmbeddings
import numpy as np

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

def similarity(a, b):
    return cosine_similarity(a, b)

embeddings_1024 = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=1024)
sentences = [sentence1, sentence2, sentence3, sentence4, sentence5]
embedded_sentences = embeddings_1024.embed_documents(sentences)
for i, sentence in enumerate(embedded_sentences):
    for j, other_sentence in enumerate(embedded_sentences):
        if i < j:
            print(
                f"[유사도 {similarity(sentence, other_sentence):.4f}] {sentences[i]} \t <=====> \t {sentences[j]}"
            )

[유사도 0.9644] 안녕하세요? 반갑습니다. 	 <=====> 	 안녕하세요? 반갑습니다!
[유사도 0.8376] 안녕하세요? 반갑습니다. 	 <=====> 	 안녕하세요? 만나서 반가워요.
[유사도 0.5042] 안녕하세요? 반갑습니다. 	 <=====> 	 Hi, nice to meet you.
[유사도 0.1362] 안녕하세요? 반갑습니다. 	 <=====> 	 I like to eat apples.
[유사도 0.8142] 안녕하세요? 반갑습니다! 	 <=====> 	 안녕하세요? 만나서 반가워요.
[유사도 0.4790] 안녕하세요? 반갑습니다! 	 <=====> 	 Hi, nice to meet you.
[유사도 0.1318] 안녕하세요? 반갑습니다! 	 <=====> 	 I like to eat apples.
[유사도 0.5128] 안녕하세요? 만나서 반가워요. 	 <=====> 	 Hi, nice to meet you.
[유사도 0.1409] 안녕하세요? 만나서 반가워요. 	 <=====> 	 I like to eat apples.
[유사도 0.2249] Hi, nice to meet you. 	 <=====> 	 I like to eat apples.


# Embedding Caching

In [2]:
from langchain.storage import LocalFileStore
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
import json

# OpenAI 임베딩을 사용하여 기본 임베딩 설정
embedding = OpenAIEmbeddings()

# 로컬 파일 저장소 설정
store = LocalFileStore("./cache/")

# 캐시를 지원하는 임베딩 생성
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=embedding,
    document_embedding_cache=store,
    namespace=embedding.model,  # 기본 임베딩과 저장소를 사용하여 캐시 지원 임베딩을 생성
)

with open(r"C:\Users\cn120\바탕 화면\rag\fairness\data\dumy\total_articls.json", 'r', encoding='UTF-8') as f:
    json_data = json.load(f)

text_splitter = CharacterTextSplitter(
    # 텍스트를 분할할 때 사용할 구분자를 지정합니다. 기본값은 "\n\n"입니다.
    separator= " ",
    # 분할된 텍스트 청크의 최대 크기(글자 갯수)를 지정합니다.
    chunk_size=3000,
    # 분할된 텍스트 청크 간의 중복되는 문자 수를 지정합니다.
    chunk_overlap=50,
    # 텍스트의 길이를 계산하는 함수를 지정합니다.
    length_function=len,
    # 구분자가 정규식인지 여부를 지정합니다.
    is_separator_regex=False,
)

# text_splitter를 사용하여 state_of_the_union 텍스트를 문서로 분할합니다.
documents = text_splitter.create_documents([json_data[i]['기사본문'] for i in range(len(json_data))])
db = FAISS.from_documents(documents, cached_embedder)

ImportError: Could not import faiss python package. Please install it with `pip install faiss-gpu` (for CUDA supported GPU) or `pip install faiss-cpu` (depending on Python version).